In [1]:
import xmltodict 
import pandas as pd
from os import listdir
import os


### COMPOSICIÓN DEL XML

<presentacion>
    <periodo>…</periodo>
    <nroPresentacion>…</nroPresentacion>
    <fechaPresentacion>…</fechaPresentacion>
    <empleado>…</empleado>
    <cargasFamilia>…</cargasFamilia>
    <ganLiqOtrosEmpEnt>…</ganLiqOtrosEmpEnt>
    <deducciones>…</deducciones>
    <retPerPagos>…</retPerPagos>
    <ajustes>…</ajustes>
    <datosAdicionales>…</datosAdicionales>
<presentacion>

Código Descripción
1 Cuotas Médico-Asistenciales
2 Primas de Seguro para el caso de muerte / riesgo de muerte
3 Donaciones
4 Intereses Préstamo Hipotecario
5 Gastos de Sepelio
7 Gastos Médicos y Paramédicos
8 Deducción del Personal Doméstico
11 Gastos de Representación e Intereses de Corredores y Viajantes de Comercio
21 Gastos de Adquisición de Indumentaria y Equipamiento para uso Exclusivo en el
22 Alquiler de Inmuebles destinados a casa habitación
23 Primas de Ahorro correspondientes a Seguros Mixtos
24 Aportes correspondientes a Planes de Seguro de Retiro Privados
32 Gastos de Educación

Deducciones que NO poseen periodos

9 Aporte a Sociedades de Garantía Recíproca
10 Vehiculos de Corredores y Viajantes de Comercio
25 Adquisición de Cuotapartes de Fondos Comunes de Inversión con fines de retiro
99 Otras Deducciones

In [71]:
def apertura_deducciones_por_mes(empleado, tipo_deduccion, mes_desde, mes_hasta, monto_mensual):
    '''
    Esta funcion devuelve un df con la cantidad de registros necesarios, segun el intervalo desde hasta (periodo) por cada deducción.
    '''
    datos_572 = [[empleado, tipo_deduccion, monto_mensual]]
    df_datos_572 = pd.DataFrame(datos_572, columns=["Cuil","Deduccion", "Importe"])

    cant_filas = (mes_hasta - mes_desde + 1)

    df_datos_572 = pd.concat([df_datos_572]*cant_filas, ignore_index=True)

    nueva_columna = []

    for i in range(14):
        if mes_desde <= i <= mes_hasta: 
            nueva_columna.append(i)

    for index, fila in df_datos_572.iterrows():
        mes = mes_desde
        if mes <= mes_hasta:
            df_datos_572["Mes"] = nueva_columna
            mes = mes_desde + 1

    return df_datos_572


In [72]:

CODIGOS_DEDUCC_MENSUAL = [1, 3, 4, 5, 8, 11, 21, 22, 32] #solamente dejar los que necesito que se procesen mensualmente

def deducciones_totales_por_mes(xml):

    empleado = xml["presentacion"]["empleado"]["cuit"]
    deducciones = xml["presentacion"]["deducciones"]["deduccion"]
    df_por_mes = pd.DataFrame([], columns=["Cuil","Deduccion", "Importe", "Mes"])
    if isinstance(deducciones, dict):
        deducciones = [deducciones]
    
    for i in deducciones:
        tipo_deduccion = int(i["@tipo"])
        if tipo_deduccion in CODIGOS_DEDUCC_MENSUAL:
            periodo = i["periodos"]["periodo"]
            
            #compruebo si es un diccionario y lo paso a listsa
            if isinstance(periodo, dict):
                periodo = [periodo]
            
            #para cada vaalor, determino los valores, ya sea que tengo una deduccion o muchas
            for j in periodo: 
                mes_desde = int(j["@mesDesde"])
                mes_hasta = int(j["@mesHasta"])
                monto_mensual = float(j["@montoMensual"])
                df = apertura_deducciones_por_mes(empleado, tipo_deduccion, mes_desde, mes_hasta, monto_mensual)
                df_por_mes = pd.concat([df_por_mes, df], ignore_index=True)            
        else:
            monto_anual = float(i["montoTotal"])
            df = apertura_deducciones_por_mes(empleado, tipo_deduccion, 13, 13, monto_anual)
            df_por_mes = pd.concat([df_por_mes, df], ignore_index=True)

    return df_por_mes


In [75]:
path = "23278673804_2022_presentacion_006.xml"

with open(path) as f:
    xml = xmltodict.parse(f.read())

def cargas_por_mes(xml):

    empleado = xml["presentacion"]["empleado"]["cuit"]
    cargas = xml["presentacion"]["cargasFamilia"]["cargaFamilia"]
    df_por_mes_cargas = pd.DataFrame([], columns=["Cuil","Deduccion", "Importe", "Mes"])
    montos_cargas = {
        1 : 19621.44, 
        3 : 9895.16, 
        30 : 9895.16, 
        31 : 19790.33, 
        32: 19790.33
    }
        
    if isinstance(cargas, dict):
        cargas = [cargas]

    for i in cargas:
        parentesco = int(i["parentesco"])
        tipo_deduccion_carga = i["parentesco"]
        porcentaje_deduccion = int(i["porcentajeDeduccion"])
        mes_desde = int(i["mesDesde"])
        mes_hasta = int(i["mesHasta"])
        monto_mensual = montos_cargas[parentesco] * porcentaje_deduccion / 100
        df = apertura_deducciones_por_mes(empleado, "C" + tipo_deduccion_carga, mes_desde, mes_hasta, monto_mensual)
        df_por_mes_cargas = pd.concat([df_por_mes_cargas, df], ignore_index=True)  

    return(df_por_mes_cargas)



In [163]:

carpeta_xml = r"S:\Rrhh\Liq-Haberes\Año 2022\Ganancias\572WEB"

listdir(carpeta_xml)
df_deducciones = pd.DataFrame() #[], columns=["Cuil","Deduccion", "Importe", "Mes"])

def deducciones_finales(carpeta):

    for a in listdir(carpeta_xml):
        path = carpeta_xml + "\\" + a

        with open(path) as f:
            xml = xmltodict.parse(f.read())

            if "deducciones" in xml["presentacion"]:
                #print(a)
                df_deducc_mes_por_empleado = deducciones_totales_por_mes(xml)
                #display(df_deducc_mes_por_empleado)
                df_deducciones = pd.concat([df_deducc_mes_por_empleado, df_deducciones], ignore_index=True)

            if "cargasFamilia" in xml["presentacion"]:
                df_cargas_por_empleado_mes = cargas_por_mes(xml)
                df_deducciones = pd.concat([df_cargas_por_empleado_mes, df_deducciones], ignore_index=True)

    return(df_deducciones)


FileNotFoundError: [WinError 53] No se ha encontrado la ruta de acceso de la red: 'S:\\Rrhh\\Liq-Haberes\\Año 2022\\Ganancias\\572WEB'

In [173]:
path = "23165012879_2022_presentacion_011.xml"

with open(path) as f:
    xml = xmltodict.parse(f.read())

empleado = xml["presentacion"]["empleado"]["cuit"]
ingresos_oe = xml["presentacion"]["ganLiqOtrosEmpEnt"]["empEnt"]["ingresosAportes"]["ingAp"]
df_otro_empleador = pd.DataFrame([], columns=["Cuil","Descripción", "Importe", "Mes"])

if isinstance(ingresos_oe, dict):
    ingresos_oe = [ingresos_oe]

for mes in ingresos_oe:
    for k in list(mes.keys()):
        if k == "@mes":
            continue 
        valor = float(mes[k])
        if valor != 0: #and valor > 12:
            datos_df_oe = [empleado, k, valor, mes["@mes"]]
            df_oe_new = pd.DataFrame([datos_df_oe], columns=["Cuil","Descripción", "Importe", "Mes"])
            df_otro_empleador = pd.concat([df_otro_empleador, df_oe_new], ignore_index=True)
display(df_otro_empleador)


,Cuil,Descripción,Importe,Mes
0,23165012879,obraSoc,5555.63,1
1,23165012879,segSoc,16101.44,1
2,23165012879,ganBrut,80507.67,1
3,23165012879,obraSoc,5555.63,2
4,23165012879,segSoc,16101.44,2
5,23165012879,ganBrut,80507.67,2
6,23165012879,obraSoc,5555.63,3
7,23165012879,segSoc,16101.44,3
8,23165012879,ganBrut,80507.67,3
9,23165012879,obraSoc,5555.63,4
